In [1]:
import torch
import math
from torch.utils import data
import matplotlib.pyplot as plt

import torch.nn.functional as F
from torch import nn
import numpy as np
from scipy.sparse import coo_matrix
import seaborn as sns


## Data Generation

In [2]:
def swiss_roll(data_size, shuffle=True, nf=2, noise_std=0, input_ch=None):

    data2d = torch.zeros(data_size, 2, 1)
    label = torch.ones(data_size, 1)
    label[math.floor(data_size / 2):, :] = 0

    r1 = torch.linspace(0, 1, math.ceil(data_size / 2))
    r2 = torch.linspace(0.2, 1.2, math.ceil(data_size / 2))
    theta = torch.linspace(0, 4 * math.pi - 4 * math.pi / math.ceil(data_size / 2), math.ceil(data_size / 2))
    data2d[0:math.ceil(data_size / 2), 0, 0] = r1 * torch.cos(theta)
    data2d[0:math.ceil(data_size / 2), 1, 0] = r1 * torch.sin(theta)
    data2d[math.floor(data_size / 2):, 0, 0] = r2 * torch.cos(theta)
    data2d[math.floor(data_size / 2):, 1, 0] = r2 * torch.sin(theta)
    if noise_std:
        for i in range(2):
            data2d[:, i, 0] = data2d[:, i, 0] + noise_std*torch.randn(data_size)

    if shuffle:
        data2d, label = _data_shuffle(data2d, label)
    
    if nf != 2:
        data2d = _data_extension(data2d, nf, input_ch)
    
    domain = [-1.2, 1.2, -1.2, 1.2]
    return data2d, label, domain

In [3]:
data_size = 8000
train_data_size = 4000
test_data_size = data_size - train_data_size
nf = 6
data_gen = swiss_roll

In [4]:
def _data_shuffle(data2d, label):
    data_size = data2d.shape[0]
    randindex = torch.randperm(data_size)
    data2d = data2d[randindex, :, :]
    label = label[randindex, :]
    return data2d, label
def _data_extension(data2d, nf, input_ch=None):
    if nf < 2:
        print("Dimension not valid")
        return
    elif nf % 2 == 1:
        print("Using odd dimension nf")
    data_size = data2d.shape[0]
    if input_ch is not None:
        # input_ch is a list of two elements. The elements indicate where the data enters.
        idx_x = input_ch[0]
        idx_y = input_ch[1]
    else:
        idx_x = 0
        idx_y = nf-1
    data2d = torch.cat((torch.zeros(data_size, idx_x-0, 1),
                        data2d[:, 0:1, :],
                        torch.zeros(data_size, idx_y-idx_x-1, 1),
                        data2d[:, 1:2, :],
                        torch.zeros(data_size, nf-1-idx_y, 1)), 1)
    return data2d

In [5]:
data2d, labels, domain = data_gen(data_size,nf=nf)

In [6]:
class Dataset(data.Dataset):

    def __len__(self):
        return len(self.list_ids)

    def __init__(self, list_ids, data_in, labels):
        self.list_ids = list_ids
        self.data = data_in
        self.labels = labels

    def __getitem__(self, index):

        idx = self.list_ids[index]

        x = self.data[idx, :, :]
        y = self.labels[idx, :]

        return x, y

partition = {'train': range(0, data_size, 2),
             'test': range(1, data_size, 2)}
training_set = Dataset(partition['train'], data2d, labels)


In [7]:
training_generator = data.DataLoader(training_set, batch_size=125, shuffle=True)

## Integrator

In [8]:
t_end = 0.5
n_layers = 8
h = t_end / n_layers

In [9]:
# # Select training parameters
alpha = 5e-4
alphac = 1e-4
learning_rate = 1e-1 #0.5e-1
max_iteration = 150
max_in_iteration = 10


# define network structure and optimizer
batch_size = 125
training_set = Dataset(partition['train'], data2d, labels)
training_generator = data.DataLoader(training_set, batch_size=batch_size, shuffle=True)

In [10]:
class H1(nn.Module):
    # Hamiltonian neural network, as presented in [1,2].
    # H_1-DNN and H_2-DNN
    # General ODE: \dot{y} = J(y,t) K(t) \tanh( K^T(t) y(t) + b(t) )
    # Constraints:
    #   J(y,t) = J_1 = [ 0 I ; -I 0 ]  or  J(y,t) = J_2 = [ 0 1 .. 1 ; -1 0 .. 1 ; .. ; -1 -1 .. 0 ].
    # Discretization method: Forward Euler
    def __init__(self, n_layers, t_end, nf, random=True, select_j='J1'):
        super().__init__()

        self.n_layers = n_layers  # nt: number of layers
        self.h = t_end / self.n_layers  #interval
        self.act = nn.Tanh()    # activation function
        self.nf = nf            # number of features

        if random:
            K = torch.randn(self.nf, self.nf, self.n_layers)
            b = torch.randn(self.nf, 1, self.n_layers)
        else:
            K = torch.ones(self.nf, self.nf, self.n_layers)
            b = torch.zeros(self.nf, 1, self.n_layers)

        self.K = nn.Parameter(K, True)
        self.b = nn.Parameter(b, True)

        if select_j == 'J1':
            j_identity = torch.eye(self.nf//2)
            j_zeros = torch.zeros(self.nf//2, self.nf//2)
            self.J = torch.cat((torch.cat((j_zeros, j_identity), 0), torch.cat((- j_identity, j_zeros), 0)), 1)
        else:
            j_aux = np.hstack((np.zeros(1), np.ones(self.nf-1)))
            J = j_aux
            for j in range(self.nf-1):
                j_aux = np.hstack((-1 * np.ones(1), j_aux[:-1]))
                J = np.vstack((J, j_aux))
            self.J = torch.tensor(J, dtype=torch.float32)

    def getK(self):
        return self.K

    def getb(self):
        return self.b

    def getJ(self):
        return self.J

    def forward(self, Y0, ini=0, end=None):

        dim = len(Y0.shape)
        Y = Y0.transpose(1, dim-1)

        if end is None:
            end = self.n_layers
        
        for j in range(ini, end):
            Y = Y + self.h * F.linear(self.act(F.linear(
                Y, self.K[:, :, j].transpose(0, 1), self.b[:, 0, j])), torch.matmul(self.J, self.K[:, :, j]))
            
        NNoutput = Y.transpose(1, dim-1)

        return NNoutput

In [11]:
class Classification(nn.Module):
    def __init__(self, nf=2, nout=1):
        super().__init__()
        self.nout = nout
        self.W = nn.Parameter(torch.zeros(self.nout, 1), True)
        self.mu = nn.Parameter(torch.zeros(1, self.nout), True)

    def forward(self, Y0):
        # Y = Y0.transpose(1, 2)
        Y = Y0.unsqueeze(1).unsqueeze(2)
        NNoutput = F.linear(Y, self.W, self.mu).squeeze(1)
        return NNoutput

In [12]:
def regularization(alpha, h, K, b):
    # Regularization function as introduced in [1]
    n_layers = K.shape[-1]
    loss = 0
    for j in range(n_layers - 1):
        loss = loss + alpha * h * (1 / 2 * torch.norm(K[:, :, j + 1] - K[:, :, j]) ** 2 +
                                   1 / 2 * torch.norm(b[:, :, j + 1] - b[:, :, j]) ** 2)
    return loss

## Training - Hamilton + Classification Layer

In [13]:
model = H1(n_layers, t_end, nf=nf, select_j='J1')

In [14]:
loss_func = nn.BCEWithLogitsLoss()
optimizer_k = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
def compute_H(y,K,b):
    n_layers = K.shape[-1]
    H = torch.sum(torch.log(torch.cosh(F.linear(
                y.squeeze(2), K[:, :, n_layers-1].transpose(0, 1), b[:, 0, n_layers-1]))),1)
    return H

In [24]:
def get_intermediate_states(model, Y0):
    # Y_out N-element list containing the intermediates states. Size of each entry: n_samples * dim2 * dim1
    # Y_out[n] = torch.zeros([batch_size, nf, 1]), with n=0,1,..,
    Y_out = [Y0]
    i = 0
    for j in range(model.n_layers):
        Y = model.forward(Y_out[j], ini=j, end=j + 1)
        Y_out.append(Y)
        Y_out[j + 1].retain_grad()
    K = model.getK()
    b = model.getb()
    
    return Y_out, K, b


In [17]:
def get_energy_with_grad(y, K, b):
    H_ALL = []
    
    n_layers = K.shape[-1]
    for layer in range(n_layers):
        dim = len(y[layer].shape)
        current_state = y[layer]
        current_state.retain_grad()
        H = torch.sum(torch.log(torch.cosh(F.linear(
                current_state.squeeze(2), K[:, :, layer].transpose(0, 1), b[:, 0, layer]))),1)
        H_ALL.append(H)
        H_ALL[layer].retain_grad()
    
    return H_ALL

In [18]:
gradient_info = True

In [19]:
if gradient_info:
        loss_func2 = nn.Identity()
        gradients_matrix = np.zeros([int(train_data_size/batch_size) * max_iteration, n_layers])
else:
        gradients_matrix = None

In [20]:
is_print = 0
for epoch in range(max_iteration):

    training_iterator = iter(training_generator)

    for i_k in range(int(data2d[partition['train']].size(0) / training_generator.batch_size)):

        local_samples, local_labels = next(training_iterator)
        
        model_c = Classification(nf=nf)
        optimizer_w = torch.optim.Adam(model_c.parameters(), lr=learning_rate)
        
        with torch.no_grad():
            YN = model(local_samples)
        
        optimizer_k.zero_grad()
        K = model.getK()
        b = model.getb()
        
        for i_w in range(max_in_iteration):  # Inner iteration
            optimizer_w.zero_grad()
            loss = loss_func(model_c(compute_H(YN,K,b)), local_labels)
            loss = loss + alphac * 0.5 * (torch.norm(model_c.W) ** 2 + torch.norm(model_c.mu) ** 2)
            loss.backward()
            optimizer_w.step()
        
        loss = loss_func(model_c(compute_H(model(local_samples),K,b)), local_labels)
        loss += regularization(alpha, h, K, b)
        loss.backward()
        li = list(optimizer_k.state)
        if not (len(li) == 0):
            for ii in range(2):
                optimizer_k.state[li[ii]]['step'] = epoch
        optimizer_k.step()
        
        ########### compute gradient ##########
        if gradient_info:
                local_samples.requires_grad = True
                
                optimizer_k.zero_grad()
                
                Y_out,K,b = get_intermediate_states(model, local_samples)
                H_ALL = get_energy_with_grad(Y_out, K, b)
                # H_ALL = get_intermediate_states(model, local_samples)
                # H_ALL = model(local_samples,is_grad=True)
                H = H_ALL[-1]
                
                loss2 = loss_func2(H.sum())
                loss2.backward()
                
                for j in range(n_layers):
                    grad = H_ALL[j].grad.numpy().sum(axis=0) / training_generator.batch_size
                    gradients_matrix[epoch * int(train_data_size / batch_size)+ i_k, j] = grad
                local_samples.requires_grad = False
        ######################################################
        ######################################################

    if epoch % 10 == 0:
        model_c = Classification(nf=nf)
        optimizer_w = torch.optim.Adam(model_c.parameters(), lr=learning_rate)
        with torch.no_grad():
            YN = model(local_samples)
        
        for i_w in range(max_in_iteration):  # Inner iteration
            optimizer_w.zero_grad()
            loss = loss_func(model_c(compute_H(YN,K,b)), local_labels)
            loss = loss + alphac * 0.5 * (torch.norm(model_c.W) ** 2 + torch.norm(model_c.mu) ** 2)
            loss.backward()
            optimizer_w.step()
            acc = (torch.ge(model_c(compute_H(model(local_samples),K,b)), 0) == local_labels).sum().numpy() / batch_size
        print('\tTrain Epoch: {:2d} - Loss: {:.6f} - Accuracy: {:.2f}%'.format(epoch, loss, acc*100))

# Train classification layer with all the data

# Accuracy results

with torch.no_grad():
    train_acc = (torch.ge(model_c(compute_H(model(data2d[partition['train'], :, :]),K,b)), 0) == labels[partition['train'], :]
                 ).sum().numpy() / train_data_size
    test_acc = (torch.ge(model_c(compute_H(model(data2d[partition['test'], :, :]),K,b)), 0) == labels[partition['test'], :]
                ).sum().numpy() / test_data_size
    
print('\tTrain Accuracy: {:.2f}%'.format(train_acc*100))
print('\tTest Accuracy: {:.2f}%'.format(test_acc*100))

AttributeError: 'NoneType' object has no attribute 'numpy'

In [21]:
Y_out[7].grad.numpy().sum(axis=0)

array([[-170.89375 ],
       [ -29.443474],
       [-208.08676 ],
       [-109.27009 ],
       [ -73.27352 ],
       [-285.6217  ]], dtype=float32)

In [22]:
H_ALL[6].grad.numpy().sum(axis=0)

AttributeError: 'NoneType' object has no attribute 'numpy'

In [23]:
H_ALL[-1].grad.numpy().sum(axis=0)

125.0